In [16]:
import os
import pdfplumber
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials

def extract_text_from_pdf(pdf_path):
    """
    Extracts text from a PDF file.
    """
    with pdfplumber.open(pdf_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

def upload_text_to_google_docs(doc_name, text_content, creds_path):
    """
    Uploads text content to Google Docs as a new document and sets sharing permissions.
    """
    # Authenticate and initialize APIs
    credentials = Credentials.from_service_account_file(
        creds_path, scopes=["https://www.googleapis.com/auth/drive"]
    )
    docs_service = build("docs", "v1", credentials=credentials)
    drive_service = build("drive", "v3", credentials=credentials)

    # Create a new Google Doc
    body = {"title": doc_name}
    doc = docs_service.documents().create(body=body).execute()
    doc_id = doc.get("documentId")

    # Append content to the document
    requests = [{"insertText": {"location": {"index": 1}, "text": text_content}}]
    docs_service.documents().batchUpdate(documentId=doc_id, body={"requests": requests}).execute()

    # Set permissions to make the document accessible
    permission_body = {
        "role": "reader",  # Options: 'reader', 'writer'
        "type": "anyone",  # Options: 'user', 'group', 'domain', 'anyone'
    }
    drive_service.permissions().create(
        fileId=doc_id, body=permission_body, fields="id"
    ).execute()

    # Construct the URL
    doc_url = f"https://docs.google.com/document/d/{doc_id}/edit"
    print(f"Uploaded and shared Google Docs: {doc_name} (URL: {doc_url})")
    return doc_url

def process_pdfs_in_folder(folder_path, creds_path):
    """
    Processes all PDFs in a folder and uploads their text content to Google Docs.
    """
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.pdf'):
            pdf_path = os.path.join(folder_path, file_name)
            doc_name = os.path.splitext(file_name)[0]

            # Extract text from the PDF
            text_content = extract_text_from_pdf(pdf_path)

            # Upload to Google Docs and get the URL
            doc_url = upload_text_to_google_docs(doc_name, text_content, creds_path)
            print(f"Document URL: {doc_url}")

if __name__ == "__main__":
    # Define folder path and credentials JSON path
    folder_path = r"C:\NISHANT\Skill Academy\Statistics\Probability Distributions & Central Limit Theorem\Probability Distributions & Central Limit Theorem"  # Replace with your folder path
    creds_path = r"C:\Users\Nishant shah\OneDrive\Desktop\Numpy\subtle-reserve-441716-r6-6e49d5b4ced4.json"  # Replace with your JSON credentials file path

    process_pdfs_in_folder(folder_path, creds_path)


Uploaded and shared Google Docs: Assignment Solution - Probability Distributions & Central Limit Theorem (URL: https://docs.google.com/document/d/1-xb5rjMYq9ofL4bvBD6KoPWAtR38Nn6rPZx1wDPxdcM/edit)
Document URL: https://docs.google.com/document/d/1-xb5rjMYq9ofL4bvBD6KoPWAtR38Nn6rPZx1wDPxdcM/edit
Uploaded and shared Google Docs: Probability Distributions & Central Limit Theorem - Notes (URL: https://docs.google.com/document/d/1fQWM-IzpGIQKIsdBBHo90k6bUQdBxejncY32ufyNvSk/edit)
Document URL: https://docs.google.com/document/d/1fQWM-IzpGIQKIsdBBHo90k6bUQdBxejncY32ufyNvSk/edit
